# Target Accrual Redemption Forward (TARF) Dual Strike

This notebook demonstrates how to access and use the functionalities of **Target Accrual Redemption Forward (TARF)** which are part of our **QPS** module within LSEG Financial Analytics SDK.

**You will be able to:**
* Define TARF instrument with basic parameters
* Configure pricing parameters
* Evaluate TARF Analytics

TARF Dual Strike - This is a variant of a regular TARF. A schedule of fixing dates is defined by its starting fixing date, expiry and frequency. The Notional Amount can be entered only in FOR and represents the total notional for the ITM side of the deal (depending on the Call/Put field). The total notional amount is divided equally on all fixing dates (unweighted) or according to user-defined weights (weighted).

**Two strikes are provided, an ITM strike and an OTM strike. For a Call direction, the ITM strike must be above or equal to the OTM strike. Conversely, for a Put direction, the ITM strike must be below or equal to the OTM strike.** When the ITM and OTM strike are equal, the instrument converges to the regular FX TARF.

For every fixing date where the fixing is ITM (relative to the ITM strike), the holder enters into a forward @ strikeITM for an amount equal to the notional amount allocated to said fixing date. Therefore, in the case of a Call, the holder earns the difference Fixing Notional x (Fixing-StrikeITM) in DOM.

For every fixing date where the fixing is OTM (relative to the OTM strike), the holder enters into a forward @ strikeOTM for a multiple (Leverage) of the allocated notional amount on said fixing date. For example, shall the notional per fixing be 1m eur and the Leverage be set at 2, the holder enters into a forward @ strikeOTM for 2m eur. Therefore, in the case of a Call, the holder loses 2 x Fixing Notional x (Fixing-StrikeOTM) in DOM. The leverage can be unique for all fixing dates or can be specifically defined for each fixing date. The leverage is valid in the interval 1-4.

For every fixing date where the fixing falls in-between the ITM and OTM strikes, no payoff is generated.

The positive side of the deal on fixing dates where the fixing is ITM is progressively cumulated. At any time the cumulated positive payoff reaches or surpasses a pre-defined target, the instrument is terminated. Hence, it is possible that the instrument may be subject to an early redemption. The target is defined in terms of conventional 'Big Figures'. The scale of of 1BF is defined on a per-ccypair basis.

An enumerator (Payment Mode) determines how to treat the cumulated payoff amount when exceeding the target. If the mode is set to 'Full Payment', the full amount is paid even for the part exceeding the target. If the mode is set to 'Capped Payment', the final payment exceeding the target is capped, so that the cumulative total earned is exactly the target.

Notice that the target conventionally does not take into account notional weights. It is defined purely as the cumulated 'market movement' on the side which is beneficial to the holder. For a simple unweighted instrument, the target also represents the amount earned by the holder. However, for a weighted instrument, the target does not represent exactly the 'profit' of the holder.

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.structured_products` - for Structured Products instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import structured_products as sp

import json
import datetime as dt
import pandas as pd
from IPython.display import display

## Data Preparation

To define a Structured Product instrument, in this example dual strike TARF, you need to follow a structured 3-step process:
* **Structured Product Definition** - Specify basic Structured Product parameters (underlying rate, payoff description, dates, notional)
* **Structured Product Instrument Definition** - Create the instrument object
* **Pricing Preferences** - Configure pricing models and parameters, optional

In [2]:
# 1. Create SP definition object

print("Step 1: Configuring instrument definition...")

TARF_definition = sp.StructuredProductsDefinition(
    deal_ccy = "EUR",
    instrument_tag = "TARF_Dual_Strike",
    inputs = [
        sp.NameTypeValue(name="Underlying", type = "string", value="USDEUR"),
        sp.NameTypeValue(name="StartDate", type = "date", value= dt.date(2022, 1, 25)),
        sp.NameTypeValue(name="EndDate", type = "string", value="StartDate + 1Y"),
        sp.NameTypeValue(name="Frequency", type = "string", value="1M"),
        sp.NameTypeValue(name="Notional", type = "string", value="10000000"),
        sp.NameTypeValue(name="LowerStrike", type = "string", value="0.85"),
        sp.NameTypeValue(name="UpperStrike", type = "string", value="0.9"),
        sp.NameTypeValue(name="Leverage", type = "string", value="2"),
        sp.NameTypeValue(name="ProfitTarget", type = "string", value="0.05"),
        sp.NameTypeValue(name="KO_Payment", type = "string", value="Settlement[t]"),
    ],
    payoff_description = [
					[
                        "ScheduleType",
                        "Schedule description",
                        "FX",
                        "Coupon",
                        "Settlement",
                        "Sum",
                        "Alive",
                        "KO_Amount",
                        "Price"
                    ],
                    [
                        "AtDate",
                        "StartDate",
                        "",
                        "",
                        "",
                        "0",
                        "1",
                        "",
                        ""
                    ],
                    [
                        "OnSchedule",
                        "DateTable(StartDate + Frequency, EndDate, Frequency, ResetGap:=0b)",
                        "FxSpot(Underlying)",
                        "If(FX[t] > UpperStrike, FX[t] - UpperStrike, If(FX[t] < LowerStrike, Leverage * (FX[t] - LowerStrike), 0))",
                        "Coupon[t] * Notional",
                        "Sum[LastDate] + Max(Coupon[t], 0)",
                        "If(Sum[t] >= ProfitTarget, 0, Alive[LastDate - 1])",
                        "Alive[LastDate - 1] * (1 - Alive[LastDate]) * KO_Payment",
                        "Alive[t] * Settlement[t] + KO_Amount[t]"
                    ],
                    [
                        "AtDate",
                        "EndDate",
                        "FxSpot(Underlying)",
                        "If(FX[t] > UpperStrike, FX[t] - UpperStrike, If(FX[t] < LowerStrike, Leverage * (FX[t] - LowerStrike), 0))",
                        "Coupon[t] * Notional",
                        "Sum[LastDate] + Max(Coupon[t], 0)",
                        "If(Sum[t] >= ProfitTarget, 0, Alive[LastDate - 1])",
                        "Alive[LastDate - 1] * (1 - Alive[LastDate]) * KO_Payment",
                        "Alive[t] * Settlement[t] + KO_Amount[t]"
                    ]
				]
)


print("	Instrument definition configured")

# 2. Create SP instrument definition object
print("Step 2: Creating instrument definition object...")

tarf_dual_strike = sp.StructuredProductsDefinitionInstrument(definition = TARF_definition)
print("	Instrument definition created")


# 3. Create SP parameters object - optional
print("Step 3: Configuring pricing parameters...")

TARF_pricing_params = sp.StructuredProductsPricingParameters(
    valuation_date= dt.date(2022, 3, 16),  # Set your desired valuation date
    numerical_method = sp.GenericNumericalMethod(method="MonteCarlo"),
    models=[sp.ModelDefinition(
            underlying_code = "USDEUR",
            underlying_tag = "USDEUR",
            underlying_currency = "EUR",
            asset_class = "ForeignExchange",
            model_name= "Dupire")]
)
print("	Pricing parameters configured")

Step 1: Configuring instrument definition...
	Instrument definition configured
Step 2: Creating instrument definition object...
	Instrument definition created
Step 3: Configuring pricing parameters...
	Pricing parameters configured


## Request Execution

In [3]:
# Execute the calculation using the price() function
# The 'definitions' parameter accepts a list of instruments definitions for batch processing

try:
    response = sp.price(
        definitions=[tarf_dual_strike],
        pricing_preferences=TARF_pricing_params,
        market_data=None,
        return_market_data=True,  # or False
        fields=None  # or specify fields as a string
    )

    errors = [a.error for a in response.data.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("Pricing execution completed")
    
except Exception as e:
    print(f"Price calculation failed: {str(e)}")
    raise

Pricing execution completed


## Results Display

### Key Sections in the `response` JSON

 - **definitions**: Instrument setup (e.g., underlying, payoff description, dates, notional), it's StructuredProductDefinition that we used. 

 - **pricingPreferences**: Valuation date, financial model, numerical method, it is StructuredProductsPricingParameters we used

 - **analytics**:
   - **tabularData**: `data`, `headers`, `statuses`
   - **cashflows**: Includes arrays and detailed `cashFlows` with discount factors
   - **description**: Instrument summary StructuredProductDefinition and also default fields not specified in the StructuredProductDefinition, but used by default in the calculation (e.g. price)
   - **greeks**: Sensitivities like `deltaAmountInDealCcy`, `gammaAmountInDealCcy`, `thetaAmountInDealCcy`, `vegaAmountInDealCcy`
   - **pricingAnalysis**: `valuationDate`, `marketDataDate`
   - **valuation**: `marketValueInDealCcy`
   - **error**: Empty if no issues

The analytics fields are useful for understanding which fields are included by default in the price function, even if they are not explicitly specified in `price` function. We detail below the description, valuation, cash-flows and greeks.

### Description

In [4]:
# Extract description from response
description = response.data.analytics[0].description

# Convert to dictionary for display
print(json.dumps(description.as_dict(), indent=4))

{
    "instrumentTag": "TARF_Dual_Strike",
    "dealCcy": "EUR",
    "discountCurveId": "IRCurve_EUREURIBORSwapZCCurve_0001-01-01T00:00:00",
    "discountCurveName": "EUR EURIBOR Swap ZC Curve",
    "outputList": {
        "Alive": 6.440199998,
        "Coupon": 0.111993891,
        "FX": 9.916816644,
        "KO_Amount": 217495.191082863,
        "Price": -336858.366742997,
        "Settlement": 1119938.906794201,
        "Sum": 1.069558125
    },
    "processingInformation": "'PricePercent' column was not retrieved in payoff description: related fields will not be retrieved. The following greeks could not be retrieved: Theta, Vega, Gamma, Delta. Make sure a 'PricePercent' column is specified in payoff description to compute greeks. "
}


### Valuation

In [5]:
# Extract vauation from the response
valuation = response.data.analytics[0].valuation

# Convert the dictionary to a DataFrame
df_tarf_valuation = pd.DataFrame(list(valuation.items()), columns=["Field", "Value"])

display(df_tarf_valuation)

,Field,Value
0,marketValueInDealCcy,-336858.366743


### Cash Flows

In [6]:
# Extract cashflows from response
cashflows = response.data.analytics[0].cashflows["cashFlows"]

# Build dataframes for all cash flow types
df_fx = pd.DataFrame(cashflows[0]['payments']).rename(columns={'amount': cashflows[0]['legTag']})
df_coupon = pd.DataFrame(cashflows[1]['payments']).rename(columns={'amount': cashflows[1]['legTag']})
df_settlement = pd.DataFrame(cashflows[2]['payments']).rename(columns={'amount': cashflows[2]['legTag']})
df_sum  = pd.DataFrame(cashflows[3]['payments']).rename(columns={'amount': cashflows[3]['legTag']})
df_alive = pd.DataFrame(cashflows[4]['payments']).rename(columns={'amount': cashflows[4]['legTag']})
df_ko_amount = pd.DataFrame(cashflows[5]['payments']).rename(columns={'amount': cashflows[5]['legTag']})
df_price = pd.DataFrame(cashflows[6]['payments']).rename(columns={'amount': cashflows[6]['legTag']})

# Merge all dataframes on the 'date' column
# Add suffix to distinguish between different data sources
combined_df = pd.concat(
    [df_fx.add_suffix('_fx'), 
     df_coupon.add_suffix('_coupon'),
     df_settlement.add_suffix('_settlement'), 
     df_sum.add_suffix('_sum'), 
     df_alive.add_suffix('_alive'), 
     df_ko_amount.add_suffix('_ko_amount'), 
     df_price.add_suffix('_price')], 
    axis=1
)

# Remove duplicate columns with the same values
combined_df = combined_df.loc[:, ~combined_df.T.duplicated()]

# Keep only one discountFactor column and sort the columns
combined_df = combined_df.loc[:,['date_fx', 'discountFactor_fx', 'FX_fx', 'Coupon_coupon', 'Settlement_settlement','Sum_sum',  'Alive_alive',
       'KO_Amount_ko_amount','Price_price', 'currency_fx', 'event_fx','occurence_fx']]

# Rename columns to match the desired layout
combined_df.columns = ['date', 'discountFactor', 'FX', 'Coupon', 'Settlement', 'Sum', 'Alive', 'KO_Amount', 'Price', 'currency', 'event', 'occurrence']

# Display the combined dataframe
display(combined_df)


,date,discountFactor,FX,Coupon,Settlement,Sum,Alive,KO_Amount,Price,currency,event,occurrence
0,2022-01-25,0.000000,0.000000,0.000000,0.000000,0.000000,1.0000,0.000000,0.000000,EUR,Payoff,Historical
1,2022-02-25,1.000000,0.887390,0.000000,0.000000,0.000000,1.0000,0.000000,0.000000,EUR,Payoff,Historical
2,2022-03-25,1.000123,0.906111,0.008875,88751.208245,0.008875,0.9992,420.491444,88751.208245,EUR,Payoff,Future
3,2022-04-25,1.000636,0.905182,0.012964,129637.598799,0.022112,0.8598,64243.933195,129067.670297,EUR,Payoff,Future
4,2022-05-25,1.001116,0.904138,0.014157,141572.664247,0.037685,0.7082,56829.976207,75729.597026,EUR,Payoff,Future
5,2022-06-27,1.001633,0.902749,0.014259,142586.795240,0.054976,0.6118,32500.727575,21453.202222,EUR,Payoff,Future
6,2022-07-25,1.002044,0.901595,0.013832,138319.229606,0.073765,0.5422,22567.736517,-15301.101480,EUR,Payoff,Future
7,2022-08-25,1.002452,0.900072,0.012399,123988.959961,0.093687,0.5042,11796.323042,-53038.889950,EUR,Payoff,Future
8,2022-09-26,1.002831,0.898346,0.010925,109254.487300,0.113801,0.4778,7255.679401,-75209.334380,EUR,Payoff,Future
9,2022-10-25,1.003093,0.896875,0.009558,95580.491240,0.133919,0.4578,4684.733257,-91524.140734,EUR,Payoff,Future


### Greeks

In [7]:
# Extract Greeks from the response
greeks = response.data.analytics[0].greeks

# Convert the dictionary to a DataFrame
df_tarf_greeks = pd.DataFrame(list(greeks.items()), columns=["Greeks", "Value"])

display(df_tarf_greeks)

,Greeks,Value
0,deltaAmountInDealCcy,303741.000
1,gammaAmountInDealCcy,-15787.500
2,vegaAmountInDealCcy,-250282.000
3,thetaAmountInDealCcy,-241.563
